In [10]:
import pandas as pd
import numpy as np
import os
import plotly.express as px

In [11]:
os.chdir('../')

In [12]:
filename = pd.read_csv('src/raw_data/ground_truth.csv')['file'][0]
recovery = pd.read_csv('src/raw_data/ground_truth.csv')['recovery'][0]
drop = pd.read_csv('src/raw_data/ground_truth.csv')['drop'][0]

df = pd.read_csv(f'src/raw_data/train/{filename}', sep = '\\s+', header = None, names = ['time', 'pressure'])
df

FileNotFoundError: [Errno 2] No such file or directory: 'src/raw_data/ground_truth.csv'

In [4]:
import ast

recovery = ast.literal_eval(recovery)
drop = ast.literal_eval(drop)

In [5]:
x1, x2 = recovery[0]

x1 = float(min(df[(df['time'] > x1 - 1) & (df['time'] < x1 + 1)]['time']))
y1 = float(df[df['time'] == x1]['pressure'])

x2 = float(min(df[(df['time'] > x2 - 1) & (df['time'] < x2 + 1)]['time']))
y2 = float(df[df['time'] == x2]['pressure'])

/var/folders/f0/c70jq9h174x32qzd077rpmy80000gn/T/ipykernel_66586/123895989.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y1 = float(df[df['time'] == x1]['pressure'])
/var/folders/f0/c70jq9h174x32qzd077rpmy80000gn/T/ipykernel_66586/123895989.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y2 = float(df[df['time'] == x2]['pressure'])


In [6]:
(x1, y1), (x2, y2)

((2419.980556, 79.022376), (2437.424444, 126.252906))

In [7]:
def log_func(x, A, B):
    return A + B * np.log(x)


B = (y2 - y1) / (np.log(x2) - np.log(x1))
A = y1 - B * np.log(x1)

x_values = np.linspace(x1, x2, 100)
y_values = log_func(x_values, A, B)

In [8]:
import plotly.graph_objects as go

def plot_data(df, selected_file, x1, y1, x2, y2):
    B = (y2 - y1) / (np.log(x2) - np.log(x1))
    A = y1 - B * np.log(x1)

    x_values = np.linspace(df["time"].min(), df["time"].max(), 200)
    x_values = x_values[x_values > 0]  
    y_values = log_func(x_values, A, B)

    fig = px.line(df, x="time", y="pressure", title=f"Visualization of {selected_file} (Reduced Data)", markers=True)
    fig.update_layout(xaxis_title="Time", yaxis_title="Pressure", template="plotly_white")

    fig.update_layout(
        xaxis=dict(rangeslider=dict(visible=True), type="linear"),
        yaxis=dict(fixedrange=False),
        legend=dict(title="Legend", x=0.99, y=0.99, xanchor="right", yanchor="top")
    )

    fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='Log Approximation', line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=[x1, x2], y=[y1, y2], mode='markers', name='Selected Points', marker=dict(color='blue', size=10)))
    
    fig.show()

In [9]:
plot_data(df, 'rofl', x1, y1, x2, y2)